In [16]:
import findspark
import numpy as np

findspark.init()
findspark.find()

# import pyspark
# findspark.find()

'/opt/spark'

In [17]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [18]:
%%time

#initialization of spark session
sk = SparkSession.builder.appName("Queries 2 on NBA").getOrCreate()

#load the datasets
ds = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")

CPU times: user 2.7 ms, sys: 537 µs, total: 3.23 ms
Wall time: 367 ms


In [19]:
#Structure of the dataset
ds.printSchema()

root
 |-- ID: string (nullable = true)
 |-- TEAM: string (nullable = true)
 |-- GP: string (nullable = true)
 |-- W: string (nullable = true)
 |-- L: string (nullable = true)
 |-- WIN%: string (nullable = true)
 |-- MIN: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FGM: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3PM: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FTM: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- OREB: string (nullable = true)
 |-- DREB: string (nullable = true)
 |-- REB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- BLKA: string (nullable = true)
 |-- PF: string (nullable = true)
 |-- PFD: string (nullable = true)
 |-- +/-: string (nullable = true)
 |-- SEASN: strin

In [20]:
ds

DataFrame[ID: string, TEAM: string, GP: string, W: string, L: string, WIN%: string, MIN: string, PTS: string, FGM: string, FGA: string, FG%: string, 3PM: string, 3PA: string, 3P%: string, FTM: string, FTA: string, FT%: string, OREB: string, DREB: string, REB: string, AST: string, TOV: string, STL: string, BLK: string, BLKA: string, PF: string, PFD: string, +/-: string, SEASN: string]

In [21]:
%%time
# Query1 
# witch team has the most field GOAL precentage? (and its win%)
# first gather all the data 
rdd = ds.rdd
rdd = rdd.map(lambda x: (x[1],x[8], x[5]))\
.sortBy(lambda x: x[1], ascending=False)
rdd.take(25)

CPU times: user 5.7 ms, sys: 7.89 ms, total: 13.6 ms
Wall time: 391 ms


[('Golden State Warriors', '44', '0.695'),
 ('New Orleans Pelicans', '43.7', '0.402'),
 ('Milwaukee Bucks', '43.4', '0.732'),
 ('Milwaukee Bucks', '43.3', '0.767'),
 ('Sacramento Kings', '43.2', '0.476'),
 ('Golden State Warriors', '43.1', '0.817'),
 ('Golden State Warriors', '42.8', '0.707'),
 ('New Orleans Pelicans', '42.7', '0.585'),
 ('Oklahoma City Thunder', '42.6', '0.598'),
 ('Los Angeles Lakers', '42.6', '0.451'),
 ('New Orleans Pelicans', '42.6', '0.417'),
 ('Golden State Warriors', '42.5', '0.89'),
 ('Memphis Grizzlies', '42.5', '0.466'),
 ('Portland Trail Blazers', '42.3', '0.646'),
 ('San Antonio Spurs', '42.3', '0.585'),
 ('Los Angeles Lakers', '42.3', '0.732'),
 ('Toronto Raptors', '42.2', '0.707'),
 ('Portland Trail Blazers', '42.2', '0.473'),
 ('San Antonio Spurs', '42.2', '0.451'),
 ('Boston Celtics', '42.1', '0.598'),
 ('Washington Wizards', '42.1', '0.39'),
 ('Indiana Pacers', '42.1', '0.616'),
 ('Denver Nuggets', '42', '0.63'),
 ('Denver Nuggets', '41.9', '0.659'),


In [22]:
%%time
# Query 2 reduce them
rdd = rdd.map(lambda x: (x[0] , [float(x[1]) , float(x[2]) , 1]))\
.reduceByKey(lambda x,y : [x[0] + y[0] , x[1] + y[1],x[2] + y[2]])\
.map(lambda x: [x[0],x[1][0] /x[1][2] ,x[1][1]])\
.sortBy(lambda x: x[1], ascending=False)
rdd.collect()

CPU times: user 29.4 ms, sys: 743 µs, total: 30.1 ms
Wall time: 352 ms


[['Golden State Warriors', 40.76, 6.139],
 ['New Orleans Pelicans', 40.32857142857143, 3.149],
 ['LA Clippers', 40.2, 3.046],
 ['San Antonio Spurs', 39.97, 6.805999999999999],
 ['Denver Nuggets', 39.779999999999994, 5.426],
 ['Oklahoma City Thunder', 39.37, 6.422000000000001],
 ['Washington Wizards', 39.06, 4.394],
 ['Portland Trail Blazers', 38.88, 5.446000000000001],
 ['Phoenix Suns', 38.88, 3.8810000000000002],
 ['Sacramento Kings', 38.79, 3.690000000000001],
 ['Los Angeles Lakers', 38.75000000000001, 4.584],
 ['Milwaukee Bucks', 38.75, 4.992999999999999],
 ['Houston Rockets', 38.61999999999999, 6.151],
 ['Minnesota Timberwolves', 38.48, 3.703],
 ['Boston Celtics', 38.42999999999999, 5.74],
 ['Dallas Mavericks', 38.4, 5.13],
 ['Toronto Raptors', 38.3, 5.6819999999999995],
 ['Atlanta Hawks', 38.290000000000006, 4.93],
 ['Indiana Pacers', 38.18, 5.6850000000000005],
 ['Philadelphia 76ers', 38.17, 4.205],
 ['Los Angeles Clippers', 38.160000000000004, 3.057],
 ['Miami Heat', 38.13, 6.02

# Answering 2 questions.
 1. Can be (the best most field GOAL) guarantee to be (the best winner)?'
 2. How about be less than average (to be in top 10)?

In [23]:

# Query 3
# 1. Can be (the best most field GOAL) guarantee to be (the best winner)?
max_field_goal_item = rdd.sortBy(lambda x: x[1], ascending=False).take(1)

max_field_goal , win_percent = max_field_goal_item[0][1] , max_field_goal_item[0][2]
print("max field goal : " + str(max_field_goal) + "\n"
      + "its max win percent : " + str(win_percent))

q1rdd = rdd.filter(lambda x: x[2] >= win_percent)\
.sortBy(lambda x: x[2], ascending=False)


if(len(q1rdd.collect()) >0 ):
    print("\nThere are some teams which their field goal is less than max field goal\nSo, it doesn't guarantee\n")
else:
     print("\nIt guarantees\n")
q1rdd.collect()


max field goal : 40.76
its max win percent : 6.139

There are some teams which their field goal is less than max field goal
So, it doesn't guarantee



[['San Antonio Spurs', 39.97, 6.805999999999999],
 ['Oklahoma City Thunder', 39.37, 6.422000000000001],
 ['Houston Rockets', 38.61999999999999, 6.151],
 ['Golden State Warriors', 40.76, 6.139]]

In [24]:

# Query 4
# 2. How about be less than average (to be in top 10)?
mean_field_goal = rdd.map(lambda x: x[1]).mean()
print("mean field goal : " + str(mean_field_goal))

q2rdd = rdd.sortBy(lambda x: x[2], ascending=False)\
.filter(lambda x: x[1] < mean_field_goal)\
.collect()

top10 = rdd.sortBy(lambda x: x[2], ascending=False)\
.take(10) #Maybe it guarantees if we decrease the amount of top (for example to change it to 3)

higher_than_mean = [higher for higher in top10 if higher in q2rdd]

if(len(higher_than_mean) >0 ):
    print("\nThere are some teams which their field goal is less than mean field goal\nSo it doesn't guarantee\n")
    print(higher_than_mean)
else:
    print("\nIt guarantees\n")
    

mean field goal : 38.30238445378153

There are some teams which their field goal is less than mean field goal
So it doesn't guarantee

[['Miami Heat', 38.13, 6.02], ['Indiana Pacers', 38.18, 5.6850000000000005], ['Toronto Raptors', 38.3, 5.6819999999999995]]


In [25]:
%%time
# Query 5
# group by data (season)
rdd = ds.rdd.cache()
rdd = rdd.map(lambda x: (x[0],x[1],x[2],x[5],x[7],x[28]))\
.groupBy(lambda x: x[5])\
.mapValues(list)

rdd.collect()

CPU times: user 34.7 ms, sys: 235 µs, total: 34.9 ms
Wall time: 324 ms


[("'10-11'",
  [('1', 'Chicago Bulls', '82', '0.756', '98.6', "'10-11'"),
   ('2', 'San Antonio Spurs', '82', '0.744', '103.7', "'10-11'"),
   ('3', 'Miami Heat', '82', '0.707', '102.1', "'10-11'"),
   ('4', 'Dallas Mavericks', '82', '0.695', '100.2', "'10-11'"),
   ('4', 'Los Angeles Lakers', '82', '0.695', '101.5', "'10-11'"),
   ('6', 'Boston Celtics', '82', '0.683', '96.5', "'10-11'"),
   ('7', 'Oklahoma City Thunder', '82', '0.671', '104.8', "'10-11'"),
   ('8', 'Orlando Magic', '82', '0.634', '99.2', "'10-11'"),
   ('9', 'Denver Nuggets', '82', '0.61', '107.5', "'10-11'"),
   ('10', 'Portland Trail Blazers', '82', '0.585', '96.3', "'10-11'"),
   ('11', 'Memphis Grizzlies', '82', '0.561', '99.9', "'10-11'"),
   ('11', 'New Orleans Hornets', '82', '0.561', '94.9', "'10-11'"),
   ('13', 'Atlanta Hawks', '82', '0.537', '95', "'10-11'"),
   ('14', 'Houston Rockets', '82', '0.524', '105.9', "'10-11'"),
   ('15', 'New York Knicks', '82', '0.512', '106.5', "'10-11'"),
   ('16', 'Philadel

# Answering 2 other question.

## Which of these 3 options for queries is faster?
#### Options 
( RDD , SQL , DATA FRAME)
#### Queries 
( SORT , SELECT , FILTER)

### SORT COMPARISON

In [78]:
%%time 
#RDD SORT
ds1 = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")
rdd = ds1.rdd
rdd = rdd.map(lambda x: (x[1],x[8]))\
.sortBy(lambda x: x[1], ascending=False)

CPU times: user 1.02 ms, sys: 3.79 ms, total: 4.81 ms
Wall time: 402 ms


In [77]:
%%time
#SQL SORT
ds2 = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")
ds2.createOrReplaceTempView("tbl")
sk.sql("""SELECT TEAM, FGM FROM tbl ORDER BY FGM ASC """)

CPU times: user 3.45 ms, sys: 0 ns, total: 3.45 ms
Wall time: 359 ms


DataFrame[TEAM: string, FGM: string]

In [80]:
%%time 
#DATA FRAME SORT
ds3 = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")
df = ds3
df = df.select("TEAM","FGM")\
.orderBy("FGM", ascending=False)

CPU times: user 5.33 ms, sys: 1.02 ms, total: 6.35 ms
Wall time: 369 ms


### FILTER COMPARISON

In [97]:
%%time
#RDD FILTER
ds1 = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")
rdd = ds1.rdd
rdd = rdd.map(lambda x: (x[1],x[8]))\
.filter(lambda x: x[1] >=35)

CPU times: user 3.2 ms, sys: 457 µs, total: 3.66 ms
Wall time: 288 ms


In [96]:
%%time
#SQL FILTER
ds2 = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")
ds2.createOrReplaceTempView("tbl")
sk.sql("""SELECT TEAM, FGM FROM tbl WHERE FGM >= 35 """)

CPU times: user 407 µs, sys: 3.65 ms, total: 4.06 ms
Wall time: 361 ms


DataFrame[TEAM: string, FGM: string]

In [95]:
%%time 
#DATA FRAME FILTER
ds3 = sk.read.format("csv").option("header", "true").load("NBA_Data/rs_10-20.csv")
df = ds3
df = df.select("TEAM","FGM")\
.where("FGM >= 35")

CPU times: user 0 ns, sys: 7.32 ms, total: 7.32 ms
Wall time: 396 ms
